In [4]:
 from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import re
import json

In [7]:
# 드라이브에 있는 CSV 파일 경로 지정
csv_file_path = '/content/drive/MyDrive/SW 경진대회/train.csv'

# CSV 파일 로드 (첫 번째 행을 열 이름으로 사용하지 않음)
df = pd.read_csv(csv_file_path, header=None, encoding='ISO-8859-1')

# 필요한 열만 추출 (인덱스, 위험도, 로그 열만)
df = df.iloc[:, 1:3]  # 앞에서 3개의 열만 선택 (인덱스, 위험도, 로그)

# 열 이름 수동 지정
df.columns = ['위험도', '로그']

# 데이터 확인
df.head(20)  # 첫 20줄 출력


<ipython-input-7-5cdca55546e7>:5: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path, header=None, encoding='ISO-8859-1')


,위험도,로그
0,level,full_log
1,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
2,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
3,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
4,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
5,1,type=SYSCALL msg=audit(1603094402.016:52981): ...
6,1,type=SYSCALL msg=audit(1611889244.855:247124):...
7,0,"Jan 22 06:28:59 localhost kibana: {""type"":""err..."
8,0,Jan 19 09:19:14 localhost logstash: [2021-01-1...
9,0,"Jan 20 08:16:43 localhost kibana: {""type"":""err..."


In [ ]:
from tqdm import tqdm
import pandas as pd
import re

# tqdm 적용을 위해 `apply` 메서드와 함께 pandas용 tqdm 적용
tqdm.pandas()

# 로그에서 네 가지 정보를 추출하는 함수 정의
def extract_log_info(log):
    # 기본값을 None으로 설정
    timestamp = None
    application = None
    log_level = None
    message = None

    # 날짜 및 시간 추출 (예: Sep 24 10:02:22)
    timestamp_match = re.search(r"(\w+ \d+ \d+:\d+:\d+)", log)
    if timestamp_match:
        timestamp = timestamp_match.group(1)

    # 애플리케이션 및 서비스 정보 추출 (예: kibana)
    application_match = re.search(r"localhost (\w+):", log)
    if application_match:
        application = application_match.group(1)

    # 로그 레벨 추출 (예: "level": "error")
    log_level_match = re.search(r'"level":\s*"(.*?)"', log)
    if log_level_match:
        log_level = log_level_match.group(1)

    # 오류 메시지 추출 (예: "message": "No Living connections")
    message_match = re.search(r'"message":\s*"(.*?)"', log)
    if message_match:
        message = message_match.group(1)

    # 네 가지 정보를 리스트로 반환
    return pd.Series([timestamp, application, log_level, message])

# 로그에서 네 가지 정보를 추출하여 새로운 컬럼 추가 (로딩바 표시)
df[['날짜 및 시간', '애플리케이션', '로그 레벨', '오류 메시지']] = df.iloc[:, 1].progress_apply(extract_log_info)

# 결과 확인 (처음 50줄 출력)
df.head(50)


100%|██████████| 472973/472973 [01:35<00:00, 4949.07it/s]


,위험도,로그,날짜 및 시간,애플리케이션,로그 레벨,오류 메시지
0,level,full_log,None,None,None,None
1,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err...",Sep 24 10:02:22,kibana,error,No Living connections
2,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...,None,logstash,None,None
3,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err...",Jan 13 01:50:40,kibana,error,No Living connections
4,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err...",None,kibana,error,No Living connections
5,1,type=SYSCALL msg=audit(1603094402.016:52981): ...,None,None,None,None
6,1,type=SYSCALL msg=audit(1611889244.855:247124):...,None,None,None,None
7,0,"Jan 22 06:28:59 localhost kibana: {""type"":""err...",Jan 22 06:28:59,kibana,error,No Living connections
8,0,Jan 19 09:19:14 localhost logstash: [2021-01-1...,Jan 19 09:19:14,logstash,None,None
9,0,"Jan 20 08:16:43 localhost kibana: {""type"":""err...",Jan 20 08:16:43,kibana,error,No Living connections


In [ ]:
from tqdm import tqdm
import pandas as pd

# 비정상 행위를 탐지하는 함수 정의
def detect_abnormal_behaviors(df):
    # 비정상 행위 로그를 저장할 리스트
    abnormal_logs = []

    # 로그 빈도 확인을 위한 딕셔너리 초기화
    error_count = {}

    # tqdm을 사용하여 iterrows에 로딩바 적용
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Detecting Abnormal Behaviors"):
        is_abnormal = False
        reason = []

        # 1. 로그 빈도 급증 탐지
        timestamp = row['날짜 및 시간']
        if timestamp in error_count:
            error_count[timestamp] += 1
        else:
            error_count[timestamp] = 1

        if error_count[timestamp] > 10:  # 같은 타임스탬프에서 10개 이상
            is_abnormal = True
            reason.append("로그 빈도 급증")

        # 2. 의도하지 않은 애플리케이션 로그 탐지
        if row['애플리케이션'] is not None and row['애플리케이션'] not in ['kibana', 'logstash', 'auditd', 'sudo']:
            is_abnormal = True
            reason.append("의도하지 않은 애플리케이션 로그")

        # 3. 로그 레벨이 'error' 또는 'fatal'일 경우
        if row['로그 레벨'] in ['error', 'fatal']:
            is_abnormal = True
            reason.append("로그 레벨 비정상")

        # 4. 심각한 에러 메시지 탐지
        if row['오류 메시지'] in ["Connection refused", "Timeout", "Database not found"]:
            is_abnormal = True
            reason.append("심각한 에러 메시지 발생")

        # 5. 새로운 오류 패턴 발생 탐지 (활성화 필요시 주석 제거)
        # if row['오류 메시지'] not in ["No Living connections", "Connection refused", "Timeout", "Database not found"]:
        #     is_abnormal = True
        #     reason.append("새로운 오류 패턴 발생")

        # 비정상 로그를 리스트에 추가
        if is_abnormal:
            row['에러 발생 원인'] = ', '.join(reason)  # 원인 열 추가
            abnormal_logs.append(row)

    # 비정상 로그가 있는 경우 새로운 DataFrame으로 반환
    return pd.DataFrame(abnormal_logs)

# 비정상 행위 탐지
abnormal_df = detect_abnormal_behaviors(df)

# 비정상 로그 수와 전체 로그 수 출력
total_logs = len(df)
abnormal_logs = len(abnormal_df)

print(f"{total_logs}개의 로그 중에 {abnormal_logs}개의 로그에서 비정상 행위가 발견되었습니다.")

# 비정상 로그 표로 출력 (필요한 열만)
abnormal_df_display = abnormal_df[['위험도', '날짜 및 시간', '애플리케이션', '로그 레벨', '오류 메시지', '에러 발생 원인']]
abnormal_df_display.reset_index(drop=True, inplace=True)
abnormal_df_display.index = abnormal_df.index  # 원래 인덱스 유지
abnormal_df_display.head(100)



Detecting Abnormal Behaviors: 100%|██████████| 472973/472973 [03:13<00:00, 2445.94it/s]


472973개의 로그 중에 286324개의 로그에서 비정상 행위가 발견되었습니다.


,위험도,날짜 및 시간,애플리케이션,로그 레벨,오류 메시지,에러 발생 원인
1,0,Sep 24 10:02:22,kibana,error,No Living connections,로그 레벨 비정상
3,0,Jan 13 01:50:40,kibana,error,No Living connections,로그 레벨 비정상
4,0,None,kibana,error,No Living connections,로그 레벨 비정상
7,0,Jan 22 06:28:59,kibana,error,No Living connections,로그 레벨 비정상
9,0,Jan 20 08:16:43,kibana,error,No Living connections,로그 레벨 비정상
...,...,...,...,...,...,...
183,0,None,kibana,error,Request Timeout after 30000ms,"로그 빈도 급증, 로그 레벨 비정상"
192,0,None,logstash,None,None,로그 빈도 급증
193,1,None,None,None,None,로그 빈도 급증
194,0,None,kibana,None,Failed to poll for work: Error: No Living conn...,로그 빈도 급증
